### Implementing FunkSVD

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('data/movies_clean.csv')
reviews = pd.read_csv('data/reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]
 [ 7.  8. 10.  8.]
 [ 9.  5.  9.  7.]
 [ 9.  8. 10.  8.]
 [ 9. 10. 10.  9.]
 [10.  9. 10.  8.]
 [ 5.  8.  5.  8.]
 [10.  8. 10. 10.]
 [ 9.  9. 10. 10.]
 [ 9.  8.  8.  8.]
 [10.  8.  1. 10.]
 [ 5.  6. 10. 10.]
 [ 8.  7. 10.  7.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the function on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of the matrix factorization.

In [22]:
array = np.ndarray(shape=(5,8), dtype=float, order='F')
array[2,4] = np.nan
np.count_nonzero(~np.isnan(array))

39

In [65]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of movies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    # helpful link: https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.rand.html
    user_mat = np.random.rand(n_users, latent_features) # user matrix filled with random values of shape user x latent 
    movie_mat = np.random.rand(latent_features, n_movies) # movie matrix filled with random values of shape latent x movies
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimization Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for i in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for u in range(n_users):
            for v in range(n_movies):
                
                # if the rating exists
                if ratings_mat[u, v] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[u, v] - np.dot(user_mat[u, :], movie_mat[:, v])
                    
                    # Keep track of the total sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[u, k] += learning_rate * (2*diff*movie_mat[k,v])
                        movie_mat[k, v] += learning_rate * (2*diff*user_mat[u,k])
                    
        # print results for iteration
        print('{:<20} {:.3f}'.format(i+1, sse_accum/num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [66]:
# use your function with 4 latent features, lr of 0.005 and 10 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=.005, iters=10)

Optimization Statistics
Iterations | Mean Squared Error 
1                    41.694
2                    14.000
3                    3.977
4                    2.783
5                    2.650
6                    2.590
7                    2.534
8                    2.475
9                    2.410
10                   2.339


In [52]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.02753605  9.10999893 10.4646153  10.16977456]
 [ 8.92504436  7.41036011  9.04025254  8.69274855]
 [ 7.8703504   7.0526004   8.31454804  7.61428419]
 [ 9.68637085  8.34920449 10.00593464  9.50105562]
 [ 8.65894136  7.10929715  9.01792851  8.61589594]
 [ 6.95626197  5.76025739  7.34886339  6.80784176]
 [ 9.1797787   8.18507151  9.53773271  9.24804385]
 [ 8.03374024  7.08193926  8.66780803  8.29639209]
 [ 8.40213822  7.38884543  8.89443926  8.56744373]
 [ 7.68121648  6.96157448  8.03409023  7.56846864]
 [ 8.80805839  7.65354579  9.19341977  9.07514502]
 [ 9.37773811  8.7219665  10.07061246  9.50422549]
 [ 9.17851277  8.25374097  9.86417214  9.10490568]
 [ 6.53418005  6.24955633  6.75172721  6.60085019]
 [ 9.82420391  8.37526273 10.01546673  9.97479571]
 [ 9.56507793  8.90305164 10.29202546  9.57703882]
 [ 8.33037078  7.42990356  8.58539781  8.26732202]
 [ 7.31473266  5.80373452  7.38318517  7.25463596]
 [ 8.38431661  7.21296963  8.89737693  8.37967545]
 [ 8.38490257  6.5784439   8.63

**Write your summary here.**

The algorithm has not converged yet but we are on the way.

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting U and V matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [76]:
# use your function with 4 latent features, lr of 0.005 and 250 iterations
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1                    42.423
2                    14.436
3                    4.253
4                    2.942
5                    2.781
6                    2.727
7                    2.691
8                    2.658
9                    2.624
10                   2.586
11                   2.545
12                   2.500
13                   2.450
14                   2.394
15                   2.332
16                   2.264
17                   2.190
18                   2.110
19                   2.023
20                   1.931
21                   1.835
22                   1.735
23                   1.632
24                   1.528
25                   1.424
26                   1.321
27                   1.221
28                   1.123
29                   1.029
30                   0.939
31                   0.855
32                   0.775
33                   0.700
34                   0.631
35                   0.

In [54]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.00000405 10.00000353 10.00000146  9.99999465]
 [10.00000188  4.00000161  9.00000063  9.99999747]
 [ 8.00001071  9.00000988 10.00000415  4.99998525]
 [ 8.9999948   7.99999446  9.99999739 10.00000748]
 [ 9.99999333  4.99999343  8.99999709  9.00000938]
 [ 5.99999892  3.99999893  9.99999952  6.00000153]
 [ 8.99999657  7.99999648  9.99999843  9.00000497]
 [10.00000994  5.00000949  9.00000412  7.9999862 ]
 [ 6.99999974  7.99999936  9.9999996   8.00000057]
 [ 8.99998134  4.99998211  8.99999224  7.000026  ]
 [ 9.00000095  8.00000091 10.00000048  7.99999883]
 [ 8.9999809   9.99998172  9.99999219  9.00002682]
 [ 9.99999547  8.99999622  9.99999872  8.00000639]
 [ 4.99999355  7.99999406  4.99999769  8.00000926]
 [10.00001076  8.00001094 10.00000517  9.99998514]
 [ 8.99999192  8.99999268  9.99999714 10.00001142]
 [ 9.00000717  8.00000727  8.00000344  7.99999014]
 [10.00000321  8.00000308  1.00000148  9.99999585]
 [ 5.00000842  6.0000083  10.00000382  9.99998846]
 [ 8.00001071  7.00001029 10.00

**Write your summary here.**

The algorithm did converge. Hurray!

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [55]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [56]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=.005, iters=250)

Optimization Statistics
Iterations | Mean Squared Error 
1                    43.311
2                    15.359
3                    4.465
4                    2.982
5                    2.816
6                    2.776
7                    2.758
8                    2.744
9                    2.732
10                   2.719
11                   2.705
12                   2.689
13                   2.672
14                   2.653
15                   2.632
16                   2.608
17                   2.581
18                   2.551
19                   2.518
20                   2.482
21                   2.442
22                   2.398
23                   2.351
24                   2.299
25                   2.244
26                   2.184
27                   2.121
28                   2.052
29                   1.980
30                   1.902
31                   1.819
32                   1.732
33                   1.638
34                   1.540
35                   1.

In [57]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.771158797757408:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 1000 users.  In the above portion, you were using a very small subset of data with no missing values.

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [80]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=.005, iters=20)

Optimization Statistics
Iterations | Mean Squared Error 
1                    23.212
2                    10.778
3                    7.385
4                    5.685
5                    4.618
6                    3.876
7                    3.327
8                    2.903
9                    2.566
10                   2.292
11                   2.065
12                   1.875
13                   1.715
14                   1.577
15                   1.460
16                   1.358
17                   1.269
18                   1.191
19                   1.123
20                   1.063


`6.` Now that you have a set of predictions for each user-movie pair,  let's answer a few questions about your results. Provide the correct values for each of the variables below, and check your solutions using the tests below.

In [81]:
# Replace each of the comments below with the correct values
num_ratings = np.count_nonzero(~np.isnan(first_1000_users)) # How many actual ratings exist in first_1000_users
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't actually have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [82]:
first_1000_users[:10]

matrix([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]])

In [83]:
print(np.dot(user_mat, movie_mat)[:10])

[[4.41600627 3.53600978 4.1174709  ... 3.53639148 2.46820431 4.29572894]
 [4.2474813  3.71529377 3.79973411 ... 3.62723215 2.23633642 4.08738721]
 [3.98966734 2.9657932  3.53040661 ... 3.21512892 2.07573168 3.90600919]
 ...
 [6.05856014 4.94659247 5.42833969 ... 5.0392309  3.23553901 5.85776527]
 [2.74308508 2.50524084 2.5184631  ... 2.49514472 1.8205267  2.45289104]
 [3.53770962 2.9017368  3.35466251 ... 2.94174652 2.27293574 3.29906565]]


In [75]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?
